## Day 24: Electromagnetic Moat

http://adventofcode.com/2017/day/24

### Part 1

I'm not sure how much time I'll have for this over the next few days but this seems straightforward enough. Represent the ports as a dictionary with keys as the opening port; there will have to be two of them, each way. Use depth-first search to find the strongest bridge. Use persistent data structures judiciously to avoid annoying mutability bugs.

In [1]:
from pyrsistent import pvector, pmap
from collections import defaultdict, deque

def get_ports(data):
    ports = defaultdict(pvector)
    
    for line in data.strip().splitlines():
        x, y = [int(f) for f in line.split('/')]
        ports[x] = ports[x].append(y)
        ports[y] = ports[y].append(x)
        
    return pmap(ports)

In [2]:
test_data = '''0/2
2/2
2/3
3/4
3/5
0/1
10/1
9/10'''

test_ports = get_ports(test_data)
test_ports

pmap({0: pvector([2, 1]), 1: pvector([0, 10]), 2: pvector([0, 2, 2, 3]), 3: pvector([2, 4, 5]), 4: pvector([3]), 5: pvector([3]), 9: pvector([10]), 10: pvector([1, 9])})

In [3]:
def strongest_bridge(ports):
    max_strength = -1
    strongest_bridge = None
    
    # Consists of (bridge, remaining ports) tuples
    # (the bridge is a pvector)
    search_space = deque()
    
    for i, port_out in enumerate(ports[0]):
        bridge = pvector([0, port_out])
        if max_strength < sum(bridge):
            strongest_bridge = bridge
            max_strength = sum(bridge)
        ports_to_search = ports.set(0, ports[0].delete(i)).set(port_out, ports[port_out].remove(0))
        search_space.append((bridge, ports_to_search))
    
    while search_space:
        bridge_so_far, ports_left = search_space.popleft()
        
        port_in = bridge_so_far[-1]
        for i, port_out in enumerate(ports_left[port_in]):
            bridge = bridge_so_far.append(port_in).append(port_out)
            if max_strength < sum(bridge):
                max_strength = sum(bridge)
                strongest_bridge = bridge
            # Actually this is a mess, using copy.deepcopy might be better
            ports_to_search = ports_left.set(port_in, ports_left[port_in].delete(i))
            ports_to_search = ports_to_search.set(port_out, ports_to_search[port_out].remove(port_in))
            search_space.append((bridge, ports_to_search))
            
    return (max_strength, strongest_bridge)

In [4]:
strongest_bridge(test_ports)

(31, pvector([0, 1, 1, 10, 10, 9]))

In [5]:
with open('input', 'r') as f:
    problem_data = f.read()
    
problem_ports = get_ports(problem_data)
problem_ports

pmap({0: pvector([47, 4, 29]), 1: pvector([8, 44, 18]), 2: pvector([5, 45]), 4: pvector([34, 32, 35, 0]), 5: pvector([12, 2]), 6: pvector([27]), 7: pvector([41, 11]), 8: pvector([28, 1, 38]), 9: pvector([24, 33, 10]), 10: pvector([28, 11, 32, 9]), 11: pvector([10, 7]), 12: pvector([35, 5]), 13: pvector([31, 26]), 14: pvector([48]), 16: pvector([19, 16, 16]), 18: pvector([1, 20, 39]), 19: pvector([16, 49, 32]), 20: pvector([29, 32, 18]), 23: pvector([37]), 24: pvector([9, 34]), 25: pvector([33]), 26: pvector([46, 13]), 27: pvector([6]), 28: pvector([10, 35, 8]), 29: pvector([20, 33, 0]), 30: pvector([37, 32]), 31: pvector([13, 37, 43]), 32: pvector([4, 30, 20, 10, 40, 37, 19]), 33: pvector([29, 9, 25, 33, 33]), 34: pvector([4, 24]), 35: pvector([12, 4, 28]), 36: pvector([38]), 37: pvector([23, 30, 31, 32]), 38: pvector([36, 8]), 39: pvector([44, 49, 18]), 40: pvector([32]), 41: pvector([48, 7]), 43: pvector([31]), 44: pvector([1, 39]), 45: pvector([47, 48, 45, 45, 2]), 46: pvector([26])

In [6]:
%time strongest_bridge(problem_ports)

CPU times: user 1min 21s, sys: 323 ms, total: 1min 21s
Wall time: 1min 21s


(1906,
 pvector([0, 47, 47, 45, 45, 45, 45, 48, 48, 41, 41, 7, 7, 11, 11, 10, 10, 32, 32, 30, 30, 37, 37, 32, 32, 20, 20, 18, 18, 1, 1, 44, 44, 39, 39, 49, 49, 49, 49, 19, 19, 32, 32, 4, 4, 34, 34, 24, 24, 9, 9, 33, 33, 33, 33, 29, 29, 0, 0, 4, 4, 35, 35, 28, 28, 8, 8, 38, 38, 36]))

### Part 2

In [7]:
def longest_bridge(ports):
    max_bridge = []
    def longer_bridge(x, y):
        return max(x, y, key=lambda b: (len(b), sum(b)))
    
    # Consists of (bridge, remaining ports) tuples
    # (the bridge is a pvector)
    search_space = deque()
    
    for i, port_out in enumerate(ports[0]):
        bridge = pvector([0, port_out])
        max_bridge = longer_bridge(bridge, max_bridge)
        ports_to_search = ports.set(0, ports[0].delete(i)).set(port_out, ports[port_out].remove(0))
        search_space.append((bridge, ports_to_search))
    
    while search_space:
        bridge_so_far, ports_left = search_space.popleft()
        
        port_in = bridge_so_far[-1]
        for i, port_out in enumerate(ports_left[port_in]):
            bridge = bridge_so_far.append(port_in).append(port_out)
            max_bridge = longer_bridge(bridge, max_bridge)
            # Actually this is a mess, using copy.deepcopy might be better
            ports_to_search = ports_left.set(port_in, ports_left[port_in].delete(i))
            ports_to_search = ports_to_search.set(port_out, ports_to_search[port_out].remove(port_in))
            search_space.append((bridge, ports_to_search))
            
    return (sum(max_bridge), len(max_bridge)//2, max_bridge)

In [8]:
longest_bridge(test_ports)

(19, 4, pvector([0, 2, 2, 2, 2, 3, 3, 5]))

In [9]:
%time longest_bridge(problem_ports)

CPU times: user 1min 31s, sys: 184 ms, total: 1min 31s
Wall time: 1min 31s


(1824,
 36,
 pvector([0, 4, 4, 35, 35, 12, 12, 5, 5, 2, 2, 45, 45, 45, 45, 48, 48, 41, 41, 7, 7, 11, 11, 10, 10, 28, 28, 8, 8, 1, 1, 44, 44, 39, 39, 49, 49, 49, 49, 19, 19, 32, 32, 20, 20, 29, 29, 33, 33, 33, 33, 9, 9, 24, 24, 34, 34, 4, 4, 32, 32, 30, 30, 37, 37, 31, 31, 13, 13, 26, 26, 46]))

That'll do.

### Post-mortem

The time taken in the solution above is unsatisfactory. Copying instead of using pyrsistent's data structures made things a lot worse, which is useful information.

In [10]:
def get_ports(data):
    ports = defaultdict(set)
    
    for key, line in enumerate(data.strip().splitlines()):
        x, y = [int(f) for f in line.split('/')]
        port = (x, y, key)
        ports[x].add(port)
        ports[y].add(port)

    return ports

In [11]:
test_components = get_ports(test_data)
test_components

defaultdict(set,
            {0: {(0, 1, 5), (0, 2, 0)},
             1: {(0, 1, 5), (10, 1, 6)},
             2: {(0, 2, 0), (2, 2, 1), (2, 3, 2)},
             3: {(2, 3, 2), (3, 4, 3), (3, 5, 4)},
             4: {(3, 4, 3)},
             5: {(3, 5, 4)},
             9: {(9, 10, 7)},
             10: {(9, 10, 7), (10, 1, 6)}})

In [12]:
def generate_bridges(components, source=0, components_used=set()):
    for c in components[source] - components_used:
        yield (c,)
        port_out = c[1] if c[0] == source else c[0]
        for b in generate_bridges(components, port_out, components_used | {c}):
            yield (c,) + b
            
def strength(bridge):
    return sum(b[0] + b[1] for b in bridge)

max(generate_bridges(test_components, 0), key=lambda x: (len(x), strength(x)))

((0, 2, 0), (2, 2, 1), (2, 3, 2), (3, 5, 4))

In [13]:
with open('input', 'r') as f:
    problem_data = f.read()
    
problem_components = get_ports(problem_data)

In [14]:
%time answer = max(generate_bridges(problem_components, 0), key=lambda x: (len(x), strength(x)))

CPU times: user 7.22 s, sys: 64 ms, total: 7.28 s
Wall time: 7.29 s


In [15]:
strength(answer)

1824

In [16]:
def generate_bridges(components, source=0, components_used=set()):
    for c in components[source] - components_used:
        components_now_used = components_used | {c}
        yield components_now_used
        port_out = c[1] if c[0] == source else c[0]
        for b in generate_bridges(components, port_out, components_now_used):
            yield b
            
def strength(bridge):
    return sum(b[0] + b[1] for b in bridge)

max(generate_bridges(test_components, 0), key=lambda x: (len(x), strength(x)))

{(0, 2, 0), (2, 2, 1), (2, 3, 2), (3, 5, 4)}

In [17]:
%time answer = max(generate_bridges(problem_components, 0), key=lambda x: (len(x), strength(x)))

CPU times: user 5.31 s, sys: 4 µs, total: 5.31 s
Wall time: 5.31 s


In [18]:
strength(answer)

1824

In [19]:
from pyrsistent import pset


def get_ports(data):
    ports = defaultdict(pset)
    
    for key, line in enumerate(data.strip().splitlines()):
        x, y = [int(f) for f in line.split('/')]
        port = (x, y, key)
        ports[x] = ports[x].add(port)
        ports[y] = ports[y].add(port)

    return ports


def generate_bridges(components, source=0, components_used=pset()):
    for c in components[source] - components_used:
        components_now_used = components_used.add(c)
        yield components_now_used
        port_out = c[1] if c[0] == source else c[0]
        for b in generate_bridges(components, port_out, components_now_used):
            yield b

In [20]:
test_components = get_ports(test_data)
max(generate_bridges(test_components, 0), key=lambda x: (len(x), strength(x)))

pset([(2, 2, 1), (0, 2, 0), (3, 5, 4), (2, 3, 2)])

In [21]:
strength(_)

19

In [22]:
problem_components = get_ports(problem_data)
%time max(generate_bridges(problem_components, 0), key=lambda x: (len(x), strength(x)))

CPU times: user 37.1 s, sys: 3.55 ms, total: 37.1 s
Wall time: 37.3 s


pset([(45, 48, 25), (19, 32, 55), (29, 33, 30), (37, 31, 12), (2, 5, 31), (32, 30, 10), (45, 45, 28), (35, 4, 15), (10, 11, 21), (20, 29, 17), (41, 48, 8), (8, 1, 26), (12, 35, 6), (33, 33, 49), (9, 24, 23), (26, 46, 44), (10, 28, 14), (0, 4, 29), (32, 20, 18), (31, 13, 0), (8, 28, 24), (37, 30, 7), (7, 41, 13), (39, 44, 42), (32, 4, 5), (26, 13, 54), (11, 7, 33), (34, 24, 41), (12, 5, 11), (33, 9, 32), (49, 39, 52), (49, 49, 2), (2, 45, 37), (49, 19, 50), (34, 4, 1), (44, 1, 35)])

In [23]:
strength(_)

1824